In [1]:
# continued from the models trained in 11_prepare_training_data_and_train.ipynb

import pandas as pd
import numpy as np
import pathlib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import warnings
import pandas as pd
import argparse
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc

from dataset_paths import *

warnings.filterwarnings('ignore')


output_version = "20241229"
# outdir = "/media/hieunguyen/HNSD_mini/outdir"
outdir = "/media/hieunguyen/HNHD01/outdir"
dataset_name = "SPIKE_IN_20250122"
mode = "all"
PROJECT = "combine_ctcandi_ichorcna"

validation_dataset_name = "Truong_new_reqest_20250321"

# for control_sample_mode in ["full", 25, 50, 75, "background_full" , "background_25", "background_50", "background_75", "background"]:
#     for input_cancer_class in ["Liver", "Lung", "Breast", "CRC", "pan_cancer"]:
#         for selected_candi_features in ["ratio_raw", "ratio_in_read"]:

control_sample_mode = "background_full"
input_cancer_class = "Liver"
selected_candi_features = "ratio_raw"

input_features = ["ichorCNA", selected_candi_features] 

path_to_main_output = os.path.join(outdir, PROJECT, output_version, dataset_name)
path_to_07_output = os.path.join(path_to_main_output, f"07_output_{mode}", input_cancer_class)
path_to_08_output = os.path.join(path_to_main_output, f"08_output_{mode}", input_cancer_class)
path_to_11_output = os.path.join(path_to_main_output, 
                                    f"11_output_{mode}", 
                                    input_cancer_class, 
                                    f"feature_{selected_candi_features}", 
                                    f"{control_sample_mode}")
path_to_12_output = os.path.join(path_to_main_output, 
                                    f"12_output_{mode}", 
                                    input_cancer_class, 
                                    f"feature_{selected_candi_features}", 
                                    f"{control_sample_mode}",
                                    validation_dataset_name)

path_to_save_models = os.path.join(path_to_11_output, "model_files")

# create the folder to save output in this analysis
os.system(f"mkdir -p {path_to_12_output}")

all_models = dict()
lr_grid = pickle.load(open(os.path.join(path_to_save_models, "LR.pkl"), "rb"))
all_models["LR"] = lr_grid

ridge_grid = pickle.load(open(os.path.join(path_to_save_models, "Ridge.pkl"), "rb"))
all_models["Ridge"] = ridge_grid

lasso_grid = pickle.load(open(os.path.join(path_to_save_models, "Lasso.pkl"), "rb"))
all_models["Lasso"] = lasso_grid

elasticnet_grid = pickle.load(open(os.path.join(path_to_save_models, "ElasticNet.pkl"), "rb"))
all_models["ElasticNet"] = elasticnet_grid

# for i in all_models.keys():
#     # Make predictions
#     y_pred_train = all_models[i].predict(X_train)


In [12]:
fulldf = pd.read_csv(os.path.join(outdir, PROJECT, output_version, validation_dataset_name,
                                  f"08_output_{mode}", input_cancer_class,
                                  "feature.csv"), index_col = [0])
resdf = pd.DataFrame(data = fulldf.SampleID.unique(), columns = ["SampleID"])
resdf["prob"] = all_models["Lasso"].predict(fulldf[input_features].to_numpy())

In [14]:
resdf["label"] = resdf["SampleID"].apply(lambda x: "Control" if x[0] == "K" else "Cancer")